Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [34]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np

In [35]:
PUZZLE_DIM = 5
action = namedtuple('Action', ['pos1', 'pos2'])

In [36]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [37]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

array([[20, 16, 12,  4, 11],
       [18,  2, 24, 21,  8],
       [15,  9,  0,  5, 10],
       [ 1,  6, 23,  7, 14],
       [22, 13,  3, 19, 17]])

## Greedy

In [38]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np

PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2'])

class NPuzzleSolver:
    def __init__(self, start_state, goal_state):
        self.start_state = start_state
        self.goal_state = goal_state
        self.n = PUZZLE_DIM
    
    def is_goal(self, state):
        return np.array_equal(state, self.goal_state)
    
    def heuristic(self, state):
        """Calculate Manhattan distance."""
        distance = 0
        for i in range(self.n):
            for j in range(self.n):
                value = state[i, j]
                if value != 0:  # Skip the empty tile
                    goal_x, goal_y = divmod(value - 1, self.n)
                    distance += abs(i - goal_x) + abs(j - goal_y)
        return distance

    def greedy_search(self):
        """Greedy Best-First Search."""
        open_list = [(self.start_state, self.heuristic(self.start_state), [])]  # (state, heuristic, path)
        closed_set = set()
        expanded_states = 0  # Counter for the number of states evaluated
        
        while open_list:
            # Sort the open list by heuristic value (ascending)
            open_list.sort(key=lambda x: x[1])
            
            # Get the state with the smallest heuristic
            current_state, _, path = open_list.pop(0)
            
            # Increment the counter
            expanded_states += 1
            
            # Check if we've reached the goal
            if self.is_goal(current_state):
                print("Goal reached!")
                return path, expanded_states  # Return the sequence of moves and cost
            
            # Add current state to the closed set
            closed_set.add(tuple(map(tuple, current_state)))  # Convert array to tuple
            
            # Expand neighbors
            for move in available_actions(current_state):
                neighbor = do_action(current_state, move)
                neighbor_tuple = tuple(map(tuple, neighbor))
                if neighbor_tuple not in closed_set:
                    open_list.append((neighbor, self.heuristic(neighbor), path + [move]))
        
        print("No solution found.")
        return None, expanded_states

# Generare uno stato casuale
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

goal = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

solver = NPuzzleSolver(state, goal)

print("Initial State:")
print(state)

solution_path, cost = solver.greedy_search()

if solution_path:
    print("Steps to solve the puzzle:")
    for step, move in enumerate(solution_path, 1):
        print(f"{step}: move tile from {move.pos2} to {move.pos1}")
    print("\nFinal State:")
    print(goal)

    # Quality and Cost
    quality = len(solution_path)
    print(f"\nQuality (number of actions in the solution): {quality}")
    print(f"Cost (total number of actions evaluated): {cost}")
    print(f"Efficiency (Quality vs Cost): {quality / cost:.4f}")


Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

Initial State:
[[ 8  6  5  2]
 [ 1 12 15  4]
 [ 9 13  7 10]
 [14  0  3 11]]
Goal reached!
Steps to solve the puzzle:
1: move tile from (2, 1) to (3, 1)
2: move tile from (1, 1) to (2, 1)
3: move tile from (0, 1) to (1, 1)
4: move tile from (0, 2) to (0, 1)
5: move tile from (0, 3) to (0, 2)
6: move tile from (1, 3) to (0, 3)
7: move tile from (1, 2) to (1, 3)
8: move tile from (2, 2) to (1, 2)
9: move tile from (3, 2) to (2, 2)
10: move tile from (3, 3) to (3, 2)
11: move tile from (2, 3) to (3, 3)
12: move tile from (1, 3) to (2, 3)
13: move tile from (1, 2) to (1, 3)
14: move tile from (2, 2) to (1, 2)
15: move tile from (3, 2) to (2, 2)
16: move tile from (3, 3) to (3, 2)
17: move tile from (2, 3) to (3, 3)
18: move tile from (2, 2) to (2, 3)
19: move tile from (2, 1) to (2, 2)
20: move tile from (1, 1) to (2, 1)
21: move tile from (0, 1) to (1, 1)
22: move tile from (0, 0) to (0, 1)
23: move tile from (1, 0) to (0, 0)
24: move tile from (1, 1) to (1, 0)
25: move tile from (0, 1) to

### A* method

In [39]:
import heapq
import numpy as np

class PuzzleState:
    def __init__(self, board, parent=None, move=0, cost=0):
        self.board = np.array(board)
        self.parent = parent
        self.move = move
        self.cost = cost
        self.heuristic = self.calculate_heuristic()
        self.total_cost = self.cost + self.heuristic

    def calculate_heuristic(self):
        # Using Manhattan distance as the heuristic
        distance = 0
        n = len(self.board)
        for i in range(n):
            for j in range(n):
                if self.board[i][j] != 0:
                    x, y = divmod(self.board[i][j] - 1, n)
                    distance += abs(x - i) + abs(y - j)
        return distance

    def get_neighbors(self):
        neighbors = []
        n = len(self.board)
        x, y = [(i, j) for i in range(n) for j in range(n) if self.board[i][j] == 0][0]
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < n and 0 <= ny < n:
                new_board = self.board.copy()
                new_board[x][y], new_board[nx][ny] = new_board[nx][ny], new_board[x][y]
                neighbors.append(PuzzleState(new_board, self, self.move + 1, self.cost + 1))
        return neighbors

    def __lt__(self, other):
        return self.total_cost < other.total_cost

    def __eq__(self, other):
        return np.array_equal(self.board, other.board)

    def __hash__(self):
        return hash(self.board.tobytes())


def a_star(start_board, goal_board):
    start_state = PuzzleState(start_board)
    goal_state = PuzzleState(goal_board)
    open_set = []
    heapq.heappush(open_set, (start_state.total_cost, start_state))
    g_score = {hash(start_state): 0}
    closed_set = set()

    while open_set:
        _, current_state = heapq.heappop(open_set)

        if np.array_equal(current_state.board, goal_state.board):
            return reconstruct_path(current_state)

        closed_set.add(hash(current_state))

        for neighbor in current_state.get_neighbors():
            neighbor_hash = hash(neighbor)
            tentative_g_score = g_score[hash(current_state)] + 1  # Each move has a cost of 1

            if neighbor_hash in closed_set and tentative_g_score >= g_score.get(neighbor_hash, float('inf')):
                continue

            if tentative_g_score < g_score.get(neighbor_hash, float('inf')):
                g_score[neighbor_hash] = tentative_g_score
                neighbor.cost = tentative_g_score
                neighbor.total_cost = neighbor.cost + neighbor.heuristic
                heapq.heappush(open_set, (neighbor.total_cost, neighbor))

    return None

def a_star_with_metrics(start_board, goal_board):
    start_state = PuzzleState(start_board)
    goal_state = PuzzleState(goal_board)
    open_set = []
    heapq.heappush(open_set, (start_state.total_cost, start_state))
    g_score = {hash(start_state): 0}
    closed_set = set()
    nodes_expanded = 0  # Numero di nodi espansi

    while open_set:
        _, current_state = heapq.heappop(open_set)

        # Controllo stato obiettivo
        if np.array_equal(current_state.board, goal_state.board):
            solution_path = reconstruct_path(current_state)
            return {
                "path": solution_path,
                "cost": len(solution_path) - 1,  # Numero di mosse
                "nodes_expanded": nodes_expanded
            }

        closed_set.add(hash(current_state))
        nodes_expanded += 1

        for neighbor in current_state.get_neighbors():
            neighbor_hash = hash(neighbor)
            tentative_g_score = g_score[hash(current_state)] + 1  # Ogni mossa costa 1

            if neighbor_hash in closed_set and tentative_g_score >= g_score.get(neighbor_hash, float('inf')):
                continue

            if tentative_g_score < g_score.get(neighbor_hash, float('inf')):
                g_score[neighbor_hash] = tentative_g_score
                neighbor.cost = tentative_g_score
                neighbor.total_cost = neighbor.cost + neighbor.heuristic
                heapq.heappush(open_set, (neighbor.total_cost, neighbor))

    return None  # Nessuna soluzione trovata



def reconstruct_path(state):
    path = []
    while state:
        path.append(state.board)
        state = state.parent
    return path[::-1]


solver = NPuzzleSolver(state, goal)

print("Initial State:")
print(state)


# solution = a_star(state, goal)
# for step in solution:
#     for row in step:
#         print(row)
#     print()

solution = a_star_with_metrics(state, goal)

if solution:
    print("Path to solution:")
    for step in solution["path"]:
        print(step)
        print()
    print(f"Cost (number of moves): {solution['cost']}")
    print(f"Nodes expanded: {solution['nodes_expanded']}")
else:
    print("No solution found.")


Initial State:
[[ 8  6  5  2]
 [ 1 12 15  4]
 [ 9 13  7 10]
 [14  0  3 11]]
Path to solution:
[[ 8  6  5  2]
 [ 1 12 15  4]
 [ 9 13  7 10]
 [14  0  3 11]]

[[ 8  6  5  2]
 [ 1 12 15  4]
 [ 9 13  7 10]
 [14  3  0 11]]

[[ 8  6  5  2]
 [ 1 12 15  4]
 [ 9 13  0 10]
 [14  3  7 11]]

[[ 8  6  5  2]
 [ 1 12  0  4]
 [ 9 13 15 10]
 [14  3  7 11]]

[[ 8  6  0  2]
 [ 1 12  5  4]
 [ 9 13 15 10]
 [14  3  7 11]]

[[ 8  0  6  2]
 [ 1 12  5  4]
 [ 9 13 15 10]
 [14  3  7 11]]

[[ 0  8  6  2]
 [ 1 12  5  4]
 [ 9 13 15 10]
 [14  3  7 11]]

[[ 1  8  6  2]
 [ 0 12  5  4]
 [ 9 13 15 10]
 [14  3  7 11]]

[[ 1  8  6  2]
 [ 9 12  5  4]
 [ 0 13 15 10]
 [14  3  7 11]]

[[ 1  8  6  2]
 [ 9 12  5  4]
 [13  0 15 10]
 [14  3  7 11]]

[[ 1  8  6  2]
 [ 9  0  5  4]
 [13 12 15 10]
 [14  3  7 11]]

[[ 1  8  6  2]
 [ 9  5  0  4]
 [13 12 15 10]
 [14  3  7 11]]

[[ 1  8  6  2]
 [ 9  5 15  4]
 [13 12  0 10]
 [14  3  7 11]]

[[ 1  8  6  2]
 [ 9  5 15  4]
 [13  0 12 10]
 [14  3  7 11]]

[[ 1  8  6  2]
 [ 9  5 15  4]
 [13  3 